In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import torchvision
import torchvision.transforms as transforms
import numpy as np
%matplotlib inline
import random
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sp
import sklearn as skl

In [ ]:
from torchvision import models

In [ ]:
USE_CUDA = torch.cuda.is_available() # GPU를 사용가능하면 True, 아니라면 False를 리턴
device = torch.device("cuda" if USE_CUDA else "cpu") # GPU 사용 가능하면 사용하고 아니면 CPU 사용
print("다음 기기로 학습합니다:", device)

다음 기기로 학습합니다: cuda


In [ ]:
import os,sys
os.getcwd()

'/content'

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
os.chdir("/gdrive/My Drive/Colab Notebooks/PyTorch/dlwpt-code-master")
path=os.getcwd()
path

'/gdrive/My Drive/Colab Notebooks/PyTorch/dlwpt-code-master'

In [ ]:
torch.random.manual_seed(0)
weights=torch.tensor([0.2126, 0.7152, 0.0722])
img_t = torch.randn(3,5,5)
batch_t = torch.randn(2,3,5,5)

print(img_t)
img_gray_naive = img_t.mean(-3) # 그 포인트의 3개의 channel의 평균 
print(img_gray_naive)
batch_gray_naive = batch_t.mean(-3) # batch 개수 만큼 나옴 
print(f'bgn =\n {batch_gray_naive}')
print(img_gray_naive.shape, batch_gray_naive.shape)

tensor([[[-1.1258, -1.1524, -0.2506, -0.4339,  0.8487],
         [ 0.6920, -0.3160, -2.1152,  0.3223, -1.2633],
         [ 0.3500,  0.3081,  0.1198,  1.2377,  1.1168],
         [-0.2473, -1.3527, -1.6959,  0.5667,  0.7935],
         [ 0.5988, -1.5551, -0.3414,  1.8530,  0.7502]],

        [[-0.5855, -0.1734,  0.1835,  1.3894,  1.5863],
         [ 0.9463, -0.8437, -0.6136,  0.0316, -0.4927],
         [ 0.2484,  0.4397,  0.1124,  0.6408,  0.4412],
         [-0.1023,  0.7924, -0.2897,  0.0525,  0.5229],
         [ 2.3022, -1.4689, -1.5867, -0.6731,  0.8728]],

        [[ 1.0554,  0.1778, -0.2303, -0.3918,  0.5433],
         [-0.3952, -0.4462,  0.7440,  1.5210,  2.3804],
         [-1.1256, -0.3170, -1.0925, -0.0852, -0.0933],
         [ 0.6871, -0.8383,  0.0185, -0.7504,  0.1854],
         [ 0.6211,  0.6382, -0.2460,  2.3025, -1.8817]]])
tensor([[-0.2187, -0.3826, -0.0991,  0.1879,  0.9928],
        [ 0.4144, -0.5353, -0.6616,  0.6249,  0.2081],
        [-0.1757,  0.1436, -0.2867,  0.5978,

In [ ]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze(-1)
unsqueezed_weights.shape

torch.Size([3, 1, 1])

In [ ]:
img_weights = (img_t * unsqueezed_weights)
batch_weights = (batch_t * unsqueezed_weights)
img_gray_weighted = img_weights.sum(-3)
batch_gray_weighted = batch_weights.sum(-3)
print(img_weights,batch_weights,img_gray_weighted,batch_gray_weighted)

tensor([[[-2.3935e-01, -2.4499e-01, -5.3273e-02, -9.2243e-02,  1.8044e-01],
         [ 1.4712e-01, -6.7184e-02, -4.4970e-01,  6.8516e-02, -2.6858e-01],
         [ 7.4406e-02,  6.5509e-02,  2.5478e-02,  2.6313e-01,  2.3743e-01],
         [-5.2571e-02, -2.8757e-01, -3.6055e-01,  1.2047e-01,  1.6870e-01],
         [ 1.2731e-01, -3.3061e-01, -7.2573e-02,  3.9395e-01,  1.5949e-01]],

        [[-4.1875e-01, -1.2401e-01,  1.3122e-01,  9.9367e-01,  1.1345e+00],
         [ 6.7679e-01, -6.0340e-01, -4.3883e-01,  2.2595e-02, -3.5236e-01],
         [ 1.7767e-01,  3.1447e-01,  8.0396e-02,  4.5829e-01,  3.1551e-01],
         [-7.3172e-02,  5.6676e-01, -2.0717e-01,  3.7553e-02,  3.7395e-01],
         [ 1.6465e+00, -1.0506e+00, -1.1348e+00, -4.8139e-01,  6.2425e-01]],

        [[ 7.6197e-02,  1.2840e-02, -1.6630e-02, -2.8285e-02,  3.9226e-02],
         [-2.8530e-02, -3.2217e-02,  5.3718e-02,  1.0981e-01,  1.7186e-01],
         [-8.1268e-02, -2.2887e-02, -7.8876e-02, -6.1510e-03, -6.7397e-03],
        

In [ ]:
img_named = img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print("img named:", img_named.shape, img_named.names)
print("batch named:", batch_named.shape, batch_named.names)

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


/usr/local/lib/python3.6/dist-packages/torch/tensor.py:605: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:840.)
  return super(Tensor, self).refine_names(names)


In [ ]:
data=torch.randint(1,100,(3,4,5))
new= data.view(-1,5*4*3)
print(data)
print(new)
data_t=data.transpose(0,2)
print(data_t.shape,'\n',data_t)

tensor([[[12, 26, 44, 79, 57],
         [18, 91, 69, 56, 64],
         [96,  9, 99, 98, 73],
         [23, 74, 15, 84,  9]],

        [[53, 99, 60, 74,  5],
         [ 5, 63, 47,  9, 45],
         [40, 11, 80, 58, 93],
         [35, 17, 78, 31, 24]],

        [[37,  6, 62, 39, 51],
         [25, 52, 91, 24, 31],
         [50, 20, 33, 95, 20],
         [ 4, 22, 32, 81, 74]]])
tensor([[12, 26, 44, 79, 57, 18, 91, 69, 56, 64, 96,  9, 99, 98, 73, 23, 74, 15,
         84,  9, 53, 99, 60, 74,  5,  5, 63, 47,  9, 45, 40, 11, 80, 58, 93, 35,
         17, 78, 31, 24, 37,  6, 62, 39, 51, 25, 52, 91, 24, 31, 50, 20, 33, 95,
         20,  4, 22, 32, 81, 74]])
torch.Size([5, 4, 3]) 
 tensor([[[12, 53, 37],
         [18,  5, 25],
         [96, 40, 50],
         [23, 35,  4]],

        [[26, 99,  6],
         [91, 63, 52],
         [ 9, 11, 20],
         [74, 17, 22]],

        [[44, 60, 62],
         [69, 47, 91],
         [99, 80, 33],
         [15, 78, 32]],

        [[79, 74, 39],
         [56,  